In [2]:
import re
import pandas as pd
import spacy
import skweak

/Users/lucelia/Documents/GitHub/experiments/members/lucelia/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#data = pd.read_csv("./CSVs/V1/REL_ADITAMENTO_CONTRATO_V1.csv", nrows=2000)
data = pd.read_csv("./CSVs/V1/REL_ADITAMENTO_CONTRATO_V1.csv", nrows=5000)

In [4]:
data

,Unnamed: 0.1,Unnamed: 0,id_dodf,tipo_rel,id_rel,anotador_rel,tipo_ent,id_ent,anotador_ent,offset,length,texto,id_ato
0,0,222,12_22.11.2018,REL_ADITAMENTO_CONTRATO,R11,anotador_19112980,EXTRATO_ADITAMENTO_CONTRATUAL,237,anotador_19112980,564894,1100,EXTRATO DO SEGUNDO TERMO ADITIVO AO CONTRATO P...,12_22.11.2018-R11
1,1,223,12_22.11.2018,REL_ADITAMENTO_CONTRATO,R11,anotador_19112980,numero_termo_aditivo,232,anotador_19112980,564905,7,SEGUNDO,12_22.11.2018-R11
2,2,224,12_22.11.2018,REL_ADITAMENTO_CONTRATO,R11,anotador_19112980,numero_contrato,233,anotador_19112980,564965,7,25/2018,12_22.11.2018-R11
3,3,225,12_22.11.2018,REL_ADITAMENTO_CONTRATO,R11,anotador_19112980,processo_gdf,234,anotador_19112980,564983,15,0050-00473/2016,12_22.11.2018-R11
4,4,226,12_22.11.2018,REL_ADITAMENTO_CONTRATO,R11,anotador_19112980,orgao_contratante,235,anotador_19112980,565040,57,SECRETARIA DE ESTADO DA SEGURANCA PUBLICA E DA...,12_22.11.2018-R11
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4995,14975,630,REL_ADITAMENTO_CONTRATO,R55,anotador_19112980,numero_contrato,1374,anotador_19112980,226524,7,73/2020,630-R55
4996,4996,14976,630,REL_ADITAMENTO_CONTRATO,R55,anotador_19112980,processo_gdf,1371,anotador_19112980,226542,22,00054-00066524/2029-92,630-R55
4997,4997,14977,630,REL_ADITAMENTO_CONTRATO,R55,anotador_19112980,objeto_aditamento_contratual,1552,anotador_19112980,226635,337,REEQUILIBRICO ECONOMICO-FINANCEIRO dos preços ...,630-R55
4998,4998,14978,630,REL_ADITAMENTO_CONTRATO,R55,anotador_19112980,data_escrito,1372,anotador_19112980,227291,10,23/12/2021,630-R55


In [5]:
#cria um novo dataset apenas com o tipo de entidade EXTATO_CONTRATO (TEXTO)
tipo_ato1 = data.loc[data['tipo_ent'] == 'EXTRATO_ADITAMENTO_CONTRATUAL'].reset_index(drop=True)
#tipo_ato1

In [6]:
#criando novo dataframe somente com os textos
df = tipo_ato1['texto'].reset_index(drop=True)
df

0      EXTRATO DO SEGUNDO TERMO ADITIVO AO CONTRATO P...
1      EXTRATO TERCEIRO TERMO ADITIVO DO CONTRATO DE ...
2      EXTRATO SEGUNDO TERMO ADITIVO DO CONTRATO DE P...
3      EXTRATO DO 3o TERMO ADITIVO AO CONTRATO No 26/...
4      EXTRATO DE ADITIVO 3 Termo Aditivo ao Contrato...
                             ...                        
649    EXTRATO DO SEGUNDO TERMO ADITIVO AO CONTRATO N...
650    EXTRATO DO SEGUNDO TERMO ADITIVO AO CONTRATO N...
651    EXTRATO DO PRIMEIRO TERMO ADITIVO AO CONTRATO ...
652    EXTRATO DO TERCEIRO TERMO ADITIVO AO CONTRATO ...
653    EXTRATO DO SEGUNDO TERMO ADITIVO AO CONTRATO N...
Name: texto, Length: 654, dtype: object

In [7]:
#df = ["EXTRATO DO CONTRATO No 11/2019 PROCESSO: 00390-00003006/2019-92. ESPECIE: Contrato de Aquisicao de Bens pelo Distrito Federal no 11/2019. PARTES: Secretaria de Estado de Desenvolvimento Urbano e Habitacao do Distrito Federal - SEDUH/DF e INDUSTRIA DE AGUA MINERAL IBIA LTDA. DO OBJETO: Aquisicao de agua potavel de mesa sem gas e garrafao retornavel, destinados ao atendimento desta pasta, conforme demais especiGcacoes e condicoes estabelecidas no Termo de Referencia Anexo I do Edital de Pregao Eletronico no 0058/2018 - SCG/SEPLAG, da Proposta da empresa INDUSTRIA DE AGUA MINERAL IBIA LTDA de 11/06/2018 e da Lei no 8.666, de 21 de junho de 1993, que passam a integrar o presente Termo. DO VALOR: R$ 11.661,00 (onze mil, seiscentos e sessenta e um reais) conforme Nota de Empenho no 2019NE00180, emitida em 18/07/2019. Dotacao Orcamentaria P.T 15.122.6001.8517.0131 - Natureza da Despesa: 3390.30 - Fonte de Recursos: 100. DO PRAZO DE VIGENCIA: O Contrato tera vigencia de 06 (seis) meses a contar do dia 07/08/2019. DATA DE ASSINATURA: 22/07/2019. SIGNATARIOS: Pelo Distrito Federal: Mateus Leandro de Oliveira, na qualidade de Secretario de Estado. Pela Contratada: Eduardo Barros de Queiroz Rodrigues, na qualidade de Socio Administrador.", "EXTRATO DO CONTRATO Nº 06/2021 Processo: 00139-00001275/2021-00. Partes: DF - ADMINISTRAÇÃO REGIONAL DO CRUZEIRO X COMERCIAL DE ALIMENTOS PARK SUL. Objeto: O presente contrato tem por objeto a contratação de empresa especializada na área de alimentação para confecção e fornecimento de 50 KG DE BOLO CONFEITADO TIPO ANIVERSÁRIO COM MASSA PÃO DE LÓ COM RECHEIO DE BABA DE MOÇA, BEIJINHO OU DOCE DE LEITE E COBERTURA DE CHANTILLY DECORADO COM ACABAMENTO, no evento de comemoração ao aniversário de 62 anos do Cruzeiro no dia 30/11/2021. Valor do Contrato: R$ 2.000,00 (Dois Mil Reais). Dotação orçamentária: Unidade Orçamentária: 190113. Programa de Trabalho: 13.392.6219.3678.0136 . Natureza da Despesa: 339039. Fonte de Recursos: 120. Nota de Empenho: 2021NE00259. Valor do Empenho Inicial: R$ 2.000,00 (Dois Mil Reais), emitida em 26/11/2021 sob o evento 400091, na Modalidade 1   Ordinário. Fundamento Legal: Art. 24 da Lei nº 8.666/1993. Data da Assinatura: 29/11/2021. Signatários: Pelo DISTRITO FEDERAL: LUIZ EDUARDO GOMES DE PAULA PESSOA, na qualidade de Administrador Regional do Cruzeiro; pela COMERCIAL DE ALIMENTOS PARK SUL: GIVANILDES DE AGUIAR MORAIS, na qualidade de Gerente Geral."] 

In [8]:
# CODIGO RESPONSAVEL PELA LIMPEZA DO TEXTO DA BASE OURO
import re

df1 =[]

for i in df:
    i = i.replace("\n", " ")
    i = re.sub('xx[a-z]{1,10}', '', i)
    #i = i.replace(r"xx[a-z]{1,10}","")
    aux = ' '.join(i.split())
    #print(aux)
    df1.append(aux)

In [9]:
# Carregando nlp do spacy em pt-br sem o ner e o lemmatizer
nlp = spacy.load('pt_core_news_sm', disable=["ner", "lemmatizer"])
docs = list(nlp.pipe(df1))

In [18]:
def processo_(doc):
        '''
        label function para extracao de processo usando regex

        parametros:
            doc: uma string respresentando o texto de um dos contratos oferecidos no vetor da base de dados
        '''
        expression = r"[P|p][R|r][O|o][C|c][E|e][S|s][S|s][O|o][\s\S].*?[s|:]?.+?(?=[0-9]).*?(\d*[^;|,|a-zA-Z]*)"    
        match = re.search(expression, str(doc))
        if match:
                    flag = 0
                    for token in doc:
                        if match.span(1)[0]+1 in range(token.idx, (token.idx+len(token))+1) and flag == 0:
                            if(doc[token.i].text == ':'):
                                start = doc[token.i+1]
                            else:
                                start = token
                            flag = 1
                        if token.idx >= match.span(1)[1] and flag == 1 and token.i > start.i:
                            if(doc[token.i-1].text in ['.', '-', '—', ':']):
                                end = doc[token.i-1]
                            else:
                                end = token
                            yield start.i, end.i, "processo_gdf"
                            break    

def data_assinatura_(doc):
        '''
        label function para extracao de data de assinatura usando regex

        parametros:
            doc: uma string respresentando o texto de um dos contratos oferecidos no vetor da base de dados
        '''
        #expression = r"([A|a][S|s][S|s][I|i][N|n][A|a][T|t][U|u][R|r][A|a].*?[\s\S])(\d{2}\/\d{2}\/\d{4}|\d{2}[\s\S]\w+[\s\S]\w+[\s\S]\w+[\s\S]\d{4}|\w{2}[\s\S]\w+[\s\S]\w+[\s\S]\w+[\s\S]\d{4}|[\s\S](\d{2}\.\d{2}\.\d{4})|\w{2}\/\d{2}\/\d{4})"
        #expression = r"([A|a][S|s][S|s][I|i][N|n][A|a][T|t][U|u][R|r][A|a].*?[\s\S].*?[s|:]?)(\d{2}\/\d{2}\/\d{4}|\d{2}[\s\S]\w+[\s\S]\w+[\s\S]\w+[\s\S]\d{4}|\w{2}[\s\S]\w+[\s\S]\w+[\s\S]\w+[\s\S]\d{4}|[\s\S](\d{2}\.\d{2}\.\d{4})|\w{2}\/\d{2}\/\d{4})"
        
        expression = r"[A|a][S|s][S|s][I|i][N|n][A|a][T|t][U|u][R|r][A|a][\s\S].+?(\d{2}\/\d{2}\/\d{4}|\d{2}[\s\S]\w+[\s\S]\w+[\s\S]\w+[\s\S]\d{4}|\w{2}[\s\S]\w+[\s\S]\w+[\s\S]\w+[\s\S]\d{4}|[\s\S](\d{2}\.\d{2}\.\d{4})|\w{2}/\d{2}\/\d{4})"

        match = re.finditer(expression, str(doc))
        if match:
            for grupo in match:
                flag = 0
                for token in doc:
                    if grupo.span(1)[0]+1 in range(token.idx, (token.idx+len(token))+1) and flag == 0:
                        if(doc[token.i].text == ':'):
                            start = doc[token.i+1]
                        else:
                            start = token
                        flag = 1
                    if token.idx >= grupo.span(1)[1] and flag == 1 and token.i > start.i:
                        if(doc[token.i-1].text in ['.', '-', '—', ':']):
                            end = doc[token.i-1]
                        else:
                            end = token
                        yield start.i, end.i, "data_escrito"
                        break               
        
def contrato_(doc):
      
        '''
        label function para extracao de contratos usando regex

        parametros:
            doc: uma string respresentando o texto de um dos contratos oferecidos no vetor da base de dados
        '''

        #expression = r"([C|c][O|o][N|n|][T|t][R|r][A|a][T|t][O|o][\s\S][N|n].*?[o|º][\s\S].*?[s|:]?)(.*\d{1,})"
        #expression = r"(([C|c][O|o][N|n|][T|t][R|r][A|a][T|t][O|o][\s\S][N|n].*?[o|º][\s\S]|[\s\S]).*?[s|:]?)(.+?(?=[0-9]).*?)"
        #expression = "r([C|c][O|o][N|n|][T|t][R|r][A|a][T|t][O|o][\s\S].*?[s|:]?.+?(?=[0-9]).*?)(\d*[^;|,|a-zA-Z]*?)"
        expression = r"([C|c][O|o][N|n|][T|t][R|r][A|a][T|t][O|o][\s\S].+?(?=[0-9]).*?|[C|c][O|o][N|n|][V|v][E|e|Ê|ê][N|n][I|i][O|o][\s\S].*?[s|:]?.+?(?=[0-9]).*?)(\d*[^;|,|a-zA-Z]*?)"
        
        match = re.search(expression, str(doc))
        #print("Teste 4")
        if match:
            flag = 0
            for token in doc:
                if match.span(2)[0]+1 in range(token.idx, (token.idx+len(token))+1) and flag == 0:
                    if(doc[token.i].text == ':'):
                        start = doc[token.i+1]
                    else:
                        start = token
                    flag = 1
                if token.idx >= match.span(2)[1] and flag == 1 and token.i > start.i:
                    if(doc[token.i-1].text in ['.', '-', '—', ':','" "']):
                        end = doc[token.i-1]
                    else:
                        end = token
                    
                    yield start.i, end.i, "numero_contrato"
                    break
        expression = r"([C|c][O|o][N|n|][T|t][R|r][A|a][T|t][O|o][\s\S][N|n][\s|o|º|.].+?(?=[0-9]).*?)(\d*[^;|,|a-zA-Z]*?)"
        
        match = re.search(expression, str(doc))
        if match:
            flag = 0
            for token in doc:
                if match.span(2)[0]+1 in range(token.idx, (token.idx+len(token))+1) and flag == 0:
                    if(doc[token.i].text == ':'):
                        start = doc[token.i+1]
                    else:
                        start = token
                    flag = 1
                if token.idx >= match.span(2)[1] and flag == 1 and token.i > start.i:
                    if(doc[token.i-1].text in ['.', '-', '—', ':','" "']):
                        end = doc[token.i-1]
                    else:
                        end = token
                    
                    yield start.i, end.i, "numero_contrato"
                    break        
        
        expression = r"([A|a][P|p][O|o|][I|i][O|o][\s\S][F|f][I|i][N|n][A|a][N|n][C|c][E|e][I|i][R|r][O|o][\s\S][A|a][O|o][\s|s][P|p][R|r][O|o][J|j][E|e][T|t][O|o][\s\S][N|n][\s|o|º|.].+?(?=[0-9]).*?)(\d*[^;|,|a-zA-Z]*?)"
        
        match = re.search(expression, str(doc))
        if match:
            flag = 0
            for token in doc:
                if match.span(2)[0]+1 in range(token.idx, (token.idx+len(token))+1) and flag == 0:
                    if(doc[token.i].text == ':'):
                        start = doc[token.i+1]
                    else:
                        start = token
                    flag = 1
                if token.idx >= match.span(2)[1] and flag == 1 and token.i > start.i:
                    if(doc[token.i-1].text in ['.', '-', '—', ':','" "']):
                        end = doc[token.i-1]
                    else:
                        end = token
                    
                    yield start.i, end.i, "numero_contrato"
                    break       

        expression = r"([T|t][E|e][R|r|][M|m][O|o][\s\S][D|d][E|e][\s\S][O|o][U|u][T|t][O|o][R|r][G|g][A|a][\s\S][E|e][\s\S][A|a][C|c][E|e][I|i][T|t][A|a][C|c|Ç|ç][A|a|Ã|ã][O|o][\s\S][N|n][\s|o|º|.].+?(?=[0-9]).*?)(\d*[^;|,|a-zA-Z]*?)"
        match = re.search(expression, str(doc))
        if match:
            flag = 0
            for token in doc:
                if match.span(2)[0]+1 in range(token.idx, (token.idx+len(token))+1) and flag == 0:
                    if(doc[token.i].text == ':'):
                        start = doc[token.i+1]
                    else:
                        start = token
                    flag = 1
                if token.idx >= match.span(2)[1] and flag == 1 and token.i > start.i:
                    if(doc[token.i-1].text in ['.', '-', '—', ':','" "']):
                        end = doc[token.i-1]
                    else:
                        end = token
                    
                    yield start.i, end.i, "numero_contrato"
                    break 
                
        expression = r"([T|t][E|e][R|r|][M|m][O|o][\s\S][D|d][E|e][\s\S][A|a][J|j][U|u][S|s][T|t][E|e][\s\S][N|n][\s|o|º|.|.o].+?(?=[0-9]).*?)(\d*[^;|,|a-zA-Z]*?)"
        match = re.search(expression, str(doc))
        if match:
            flag = 0
            for token in doc:
                if match.span(2)[0]+1 in range(token.idx, (token.idx+len(token))+1) and flag == 0:
                    if(doc[token.i].text == ':'):
                        start = doc[token.i+1]
                    else:
                        start = token
                    flag = 1
                if token.idx >= match.span(2)[1] and flag == 1 and token.i > start.i:
                    if(doc[token.i-1].text in ['.', '-', '—', ':','" "']):
                        end = doc[token.i-1]
                    else:
                        end = token
                    
                    yield start.i, end.i, "numero_contrato"
                    break 
                        
detec_contrato = skweak.heuristics.FunctionAnnotator("detec_contrato", contrato_)
docs = list(detec_contrato.pipe(docs))

detec_processo = skweak.heuristics.FunctionAnnotator("detec_processo", processo_)
docs = list(detec_processo.pipe(docs))
 
detec_data = skweak.heuristics.FunctionAnnotator("detec_data", data_assinatura_)
docs = list(detec_data.pipe(docs))


all_fun = ["detec_contrato","detec_processo","detec_data"]
#all_fun = ["detec_contrato","detec_data"]


for i in range(len(docs)):
    skweak.utils.display_entities(docs[i], all_fun)

: 